In [1]:
import pandas as pd 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold

df = pd.read_csv('Data/Dataset.csv', decimal =".", thousands=",")
df = df.drop(['Kota','Paslon 1','Paslon 2','Total'],axis=1)

scaler = MinMaxScaler()

df_minmax = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)
dfX, dfY = df_minmax.iloc[:, :-1], df_minmax.iloc[:, [-1]]
kf = RepeatedKFold(n_splits=10, n_repeats = 3 , random_state=1)
kf.get_n_splits(dfX)
y = df['Partisipasi']


def beep():
    import IPython
    IPython.display.Audio("beep.mp3", autoplay=True)

def flatten(l):
    return [item for sublist in l for item in sublist]

In [7]:
from catboost import CatBoostRegressor
from catboost import Pool
import optuna
from sklearn.metrics import mean_squared_error
import Utils as utils

def objective(trial):
    data, target = dfX, y
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2)
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.25)
    train_pool = Pool(train_x, train_y)
    val_pool = Pool(val_x, val_y)
    test_pool = Pool(test_x)

    params = {
        "objective": trial.suggest_categorical("objective", ["RMSE"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
         "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    model = CatBoostRegressor(**params)

    pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")
    model.fit(train_pool, eval_set=val_pool, callbacks=[pruning_callback], verbose=500, early_stopping_rounds=100)

    pruning_callback.check_pruned()

    preds = model.predict(test_pool)

    results = utils.error_rate("Catboost",preds,test_y)
    print(results)

    rmse = mean_squared_error(test_y, preds, squared=False)
    return rmse


study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner(n_warmup_steps=10))

study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


    


    






[I 2023-06-07 20:12:20,347] A new study created in memory with name: no-name-1f5096c1-7af5-48ed-a82d-ebf1bc8438f1
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_32688\3008741279.py:33: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")


Learning rate set to 0.032991
0:	learn: 0.0480222	test: 0.0526873	best: 0.0526873 (0)	total: 2.35ms	remaining: 2.35s
500:	learn: 0.0352131	test: 0.0473180	best: 0.0473180 (500)	total: 579ms	remaining: 576ms


[I 2023-06-07 20:12:22,229] Trial 0 finished with value: 0.0349815838846568 and parameters: {'objective': 'RMSE', 'colsample_bylevel': 0.019294785961930627, 'depth': 6, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 4.6044011355658085}. Best is trial 0 with value: 0.0349815838846568.


Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.04686541213
bestIteration = 596

Shrink model to first 597 iterations.
('Catboost', 0.0349815838846568, 0.027204473841132337, 0.4295501324368346)
Learning rate set to 0.032991
0:	learn: 0.0491757	test: 0.0445211	best: 0.0445211 (0)	total: 249us	remaining: 249ms


C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_32688\3008741279.py:33: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")


...

0:	learn: 0.0510492	test: 0.0538492	best: 0.0538492 (0)	total: 242us	remaining: 243ms

bestTest = 0.05277874674
bestIteration = 10

Shrink model to first 11 iterations.
Learning rate set to 0.032991
0:	learn: 0.0489718	test: 0.0520263	best: 0.0520263 (0)	total: 281us	remaining: 282ms

bestTest = 0.05202625615
bestIteration = 0

Shrink model to first 1 iterations.
Learning rate set to 0.032991
0:	learn: 0.0458405	test: 0.0536157	best: 0.0536157 (0)	total: 240us	remaining: 240ms

bestTest = 0.05361573074
bestIteration = 1

Shrink model to first 2 iterations.


[I 2023-06-07 20:12:46,878] Trial 97 pruned. Trial was pruned at iteration 10.
[I 2023-06-07 20:12:46,997] Trial 98 pruned. Trial was pruned at iteration 10.


Learning rate set to 0.032991
0:	learn: 0.0475042	test: 0.0443993	best: 0.0443993 (0)	total: 5.14ms	remaining: 5.14s

bestTest = 0.04325713696
bestIteration = 10

Shrink model to first 11 iterations.
Learning rate set to 0.032991
0:	learn: 0.0550404	test: 0.0359751	best: 0.0359751 (0)	total: 270us	remaining: 271ms


[I 2023-06-07 20:12:47,161] Trial 99 pruned. Trial was pruned at iteration 95.



bestTest = 0.03364072166
bestIteration = 89

Shrink model to first 90 iterations.
Number of finished trials: 100
Best trial:
  Value: 0.0349815838846568
  Params: 
    objective: RMSE
    colsample_bylevel: 0.019294785961930627
    depth: 6
    boosting_type: Ordered
    bootstrap_type: Bayesian
    bagging_temperature: 4.6044011355658085


In [8]:
from catboost import CatBoostRegressor
from catboost import Pool
import optuna
from sklearn.metrics import mean_squared_error
import Utils as utils

def objective(trial):
    cols = ['Umur', 'Umur Squared', 'APM SMA', 'Pengeluaran Bulanan','Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']
    data, target = dfX[cols], y
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2)
    train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.25)
    train_pool = Pool(train_x, train_y)
    val_pool = Pool(val_x, val_y)
    test_pool = Pool(test_x)

    params = {
        "objective": trial.suggest_categorical("objective", ["RMSE"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
         "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    model = CatBoostRegressor(**params)

    pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")
    model.fit(train_pool, eval_set=val_pool, callbacks=[pruning_callback], verbose=500, early_stopping_rounds=100)

    pruning_callback.check_pruned()

    preds = model.predict(test_pool)

    results = utils.error_rate("Catboost",preds,test_y)
    print(results)

    rmse = mean_squared_error(test_y, preds, squared=False)
    return rmse



study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner(n_warmup_steps=10))

study.optimize(objective, n_trials=100, timeout=600)

print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


    


    






[I 2023-06-07 20:14:59,108] A new study created in memory with name: no-name-e719ca62-4c82-45dd-8d65-6e076a7b18e0
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_32688\1628479321.py:34: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")


Learning rate set to 0.032991
0:	learn: 0.0486389	test: 0.0492778	best: 0.0492778 (0)	total: 977us	remaining: 976ms
500:	learn: 0.0322777	test: 0.0391103	best: 0.0390906 (491)	total: 518ms	remaining: 516ms


[I 2023-06-07 20:15:02,012] Trial 0 finished with value: 0.036636711002973305 and parameters: {'objective': 'RMSE', 'colsample_bylevel': 0.052503740365612624, 'depth': 9, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS'}. Best is trial 0 with value: 0.036636711002973305.


999:	learn: 0.0268657	test: 0.0361930	best: 0.0361658 (958)	total: 1.06s	remaining: 0us

bestTest = 0.03616579889
bestIteration = 958

Shrink model to first 959 iterations.
('Catboost', 0.036636711002973305, 0.028105386558936983, 0.40950852705839)
Learning rate set to 0.032991
0:	learn: 0.0453744	test: 0.0597449	best: 0.0597449 (0)	total: 267us	remaining: 267ms


C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_32688\1628479321.py:34: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")


...

Learning rate set to 0.032991
0:	learn: 0.0484320	test: 0.0412963	best: 0.0412963 (0)	total: 1.28ms	remaining: 1.28s

bestTest = 0.04119179183
bestIteration = 10

Shrink model to first 11 iterations.
Learning rate set to 0.032991
0:	learn: 0.0492385	test: 0.0512994	best: 0.0512994 (0)	total: 308us	remaining: 309ms

bestTest = 0.0506999177
bestIteration = 6

Shrink model to first 7 iterations.
Learning rate set to 0.032991
0:	learn: 0.0496944	test: 0.0426903	best: 0.0426903 (0)	total: 1.28ms	remaining: 1.28s


[I 2023-06-07 20:15:47,810] Trial 96 pruned. Trial was pruned at iteration 10.
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_32688\1628479321.py:34: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")
[I 2023-06-07 20:15:47,893] Trial 97 pruned. Trial was pruned at iteration 10.
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_32688\1628479321.py:34: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")
[I 2023-06-07 20:15:48,001] Trial 98 pruned. Trial was pruned at iteration 10.



bestTest = 0.04199504562
bestIteration = 10

Shrink model to first 11 iterations.
Learning rate set to 0.032991
0:	learn: 0.0455415	test: 0.0541023	best: 0.0541023 (0)	total: 302us	remaining: 302ms

bestTest = 0.05316446872
bestIteration = 10

Shrink model to first 11 iterations.
Learning rate set to 0.032991
0:	learn: 0.0444556	test: 0.0510119	best: 0.0510119 (0)	total: 1.15ms	remaining: 1.15s

bestTest = 0.0509160874
bestIteration = 6

Shrink model to first 7 iterations.


C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_32688\1628479321.py:34: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = optuna.integration.CatBoostPruningCallback(trial, "RMSE")
[I 2023-06-07 20:15:48,105] Trial 99 pruned. Trial was pruned at iteration 10.


Learning rate set to 0.032991
0:	learn: 0.0480794	test: 0.0538179	best: 0.0538179 (0)	total: 1.21ms	remaining: 1.21s

bestTest = 0.05374895923
bestIteration = 8

Shrink model to first 9 iterations.
Number of finished trials: 100
Best trial:
  Value: 0.036636711002973305
  Params: 
    objective: RMSE
    colsample_bylevel: 0.052503740365612624
    depth: 9
    boosting_type: Ordered
    bootstrap_type: MVS


In [9]:
cols = ['Umur', 'Umur Squared', 'APM SMA', 'Pengeluaran Bulanan','Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']
utils.evaluate_catboost(dfX[cols],y,kf)

('Catboost', 0.03512429844587696, 0.028615520062230004, 0.47568209883632273)

In [11]:


def evaluate_catboost(dfX,y,kf):
  from catboost import CatBoostRegressor
  from catboost import Pool
  from sklearn.metrics import r2_score, mean_squared_error
  vals = []
  preds = []
  error_rates_dt = []

  cbr = CatBoostRegressor(
    boosting_type= 'Ordered',
    colsample_bylevel= 0.052503740365612624,
    depth= 9,
    bootstrap_type= 'MVS',
    )
  for train_index, test_index in kf.split(dfX):
    # print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = dfX.iloc[train_index], dfX.iloc[test_index]
      y_train, y_test = y.iloc[train_index], y.iloc[test_index]
      train_pool = Pool(data=X_train, label=y_train)
      test_pool = Pool(data=X_test, label=y_test)

      cbr.fit(train_pool, verbose=False)
      cbr_pred = cbr.predict(test_pool)
      preds.append(cbr_pred)
      vals.append(y_test)

  cat_df = pd.DataFrame({'Prediction':utils.flatten(preds),'Actual Values': utils.flatten(vals)})
  return utils.error_rate("Catboost",cat_df['Prediction'],cat_df['Actual Values'])

evaluate_catboost(dfX[cols],y,kf)

('Catboost', 0.0379735377563578, 0.030907234780759615, 0.3871679255729855)

Shrink model to first 90 iterations.
Number of finished trials: 100
Best trial:
  Value: 0.0349815838846568
  Params: 
    objective: RMSE
    colsample_bylevel: 0.019294785961930627
    depth: 6
    boosting_type: Ordered
    bootstrap_type: Bayesian
    bagging_temperature: 4.6044011355658085

In [12]:


def evaluate_catboost(dfX,y,kf):
  from catboost import CatBoostRegressor
  from catboost import Pool
  from sklearn.metrics import r2_score, mean_squared_error
  vals = []
  preds = []
  error_rates_dt = []

  cbr = CatBoostRegressor(
    boosting_type= 'Ordered',
    colsample_bylevel= 0.019294785961930627,
    depth= 6,
    bootstrap_type= 'Bayesian',
    bagging_temperature= 4.6044011355658085,
    )
  for train_index, test_index in kf.split(dfX):
    # print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = dfX.iloc[train_index], dfX.iloc[test_index]
      y_train, y_test = y.iloc[train_index], y.iloc[test_index]
      train_pool = Pool(data=X_train, label=y_train)
      test_pool = Pool(data=X_test, label=y_test)

      cbr.fit(train_pool, verbose=False)
      cbr_pred = cbr.predict(test_pool)
      preds.append(cbr_pred)
      vals.append(y_test)

  cat_df = pd.DataFrame({'Prediction':utils.flatten(preds),'Actual Values': utils.flatten(vals)})
  return utils.error_rate("Catboost",cat_df['Prediction'],cat_df['Actual Values'])

evaluate_catboost(dfX[cols],y,kf)

('Catboost', 0.04339805479488252, 0.03453902698156212, 0.19957637991715627)

In [14]:
model = CatBoostRegressor(verbose=1000)
grid = {
    'learning_rate': [0.03, 0.05, 0.07, 0.1],
    'depth': [4, 6, 10, 12],
    'l2_leaf_reg': [1, 3, 5, 7, 9]
}
grid_search_result = model.grid_search(grid, X=dfX[cols], y=y, plot=True, cv=kf, verbose=False)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.7794596	test: 0.8047215	best: 0.8047215 (0)	total: 704us	remaining: 704ms
999:	learn: 0.0017389	test: 0.0857531	best: 0.0857467 (985)	total: 600ms	remaining: 0us

bestTest = 0.08574666853
bestIteration = 985

0:	learn: 0.7657917	test: 0.7909031	best: 0.7909031 (0)	total: 519us	remaining: 519ms
999:	learn: 0.0004218	test: 0.0934371	best: 0.0934209 (979)	total: 753ms	remaining: 0us

bestTest = 0.09342090662
bestIteration = 979

0:	learn: 0.7521271	test: 0.7770863	best: 0.7770863 (0)	total: 654us	remaining: 654ms
999:	learn: 0.0000640	test: 0.0936871	best: 0.0936867 (995)	total: 688ms	remaining: 0us

bestTest = 0.09368672517
bestIteration = 995

0:	learn: 0.7316370	test: 0.7563645	best: 0.7563645 (0)	total: 476us	remaining: 476ms
999:	learn: 0.0000073	test: 0.0916838	best: 0.0916838 (999)	total: 648ms	remaining: 0us

bestTest = 0.0916838414
bestIteration = 999

0:	learn: 0.7836079	test: 0.8090148	best: 0.8090148 (0)	total: 781us	remaining: 781ms
999:	learn: 0.0037578	test: 0.0

In [17]:
grid_search_result['params']

{'depth': 4, 'l2_leaf_reg': 1, 'learning_rate': 0.03}

In [18]:


def evaluate_catboost(dfX,y,kf):
  from catboost import CatBoostRegressor
  from catboost import Pool
  from sklearn.metrics import r2_score, mean_squared_error
  vals = []
  preds = []
  error_rates_dt = []

  cbr = CatBoostRegressor(
    depth= 4,
    l2_leaf_reg= 1,
    learning_rate= 0.03,
    )
  for train_index, test_index in kf.split(dfX):
    # print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = dfX.iloc[train_index], dfX.iloc[test_index]
      y_train, y_test = y.iloc[train_index], y.iloc[test_index]
      train_pool = Pool(data=X_train, label=y_train)
      test_pool = Pool(data=X_test, label=y_test)

      cbr.fit(train_pool, verbose=False)
      cbr_pred = cbr.predict(test_pool)
      preds.append(cbr_pred)
      vals.append(y_test)

  cat_df = pd.DataFrame({'Prediction':utils.flatten(preds),'Actual Values': utils.flatten(vals)})
  return utils.error_rate("Catboost",cat_df['Prediction'],cat_df['Actual Values'])

evaluate_catboost(dfX[cols],y,kf)

('Catboost', 0.03603586259414829, 0.029358332918660912, 0.4481142088724709)

In [7]:
from catboost import CatBoostRegressor
from catboost import Pool
import optuna
import Utils as utils
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from optuna.integration import CatBoostPruningCallback


def objective(trial):
    cols = ['Umur', 'Umur Squared', 'APM SMA', 'Pengeluaran Bulanan','Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']
    usedDF = dfX[cols]
    # data, target = dfX[cols], y
    # train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2)
    # train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.25)
    # train_pool = Pool(train_x, train_y)
    # val_pool = Pool(val_x, val_y)
    # test_pool = Pool(test_x)

    preds = []
    vals = []

    params = {
        "objective": trial.suggest_categorical("objective", ["RMSE"]),
        "learning_rate": trial.suggest_discrete_uniform("learning_rate", 0.001, 0.05, 0.005),
        "depth": trial.suggest_int("depth", 8, 16),
        "l2_leaf_reg": trial.suggest_discrete_uniform("l2_leaf_reg", 1, 5.5, 0.5),
        "use_best_model": True,
        "random_seed": 0,
    }

    print(params)

    # if params["bootstrap_type"] == "Bayesian":
    #     params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    # elif params["bootstrap_type"] == "Bernoulli":
    #     params["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    model = CatBoostRegressor(**params, verbose=False)

    # pruning_callback = CatBoostPruningCallback(trial, "RMSE")

    for i, (train_index , test_index) in enumerate(kf.split(usedDF)):
        train_x, test_x = usedDF.iloc[train_index], usedDF.iloc[test_index]
        train_y, test_y = y.iloc[train_index], y.iloc[test_index]
        
        train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.25)
        train_pool = Pool(train_x, train_y)
        val_pool = Pool(val_x, val_y)
        test_pool = Pool(test_x)

        model.fit(train_pool, eval_set=val_pool)

        pred = model.predict(test_pool)
        # convert np float array to list

        preds.append(pred.tolist())
        vals.append(test_y.values)
        
    
    preds = flatten(preds)
    vals = flatten(vals)
    
    results = utils.error_rate("Catboost",preds ,vals)
    print(results)

    rmse = mean_squared_error(vals, preds, squared=False)
    return rmse



study = optuna.create_study(direction="minimize", pruner=optuna.pruners.MedianPruner(n_warmup_steps=10))

study.optimize(objective, n_trials=15)

print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))




[I 2023-06-08 14:38:08,042] A new study created in memory with name: no-name-a820b070-628f-4db5-9c86-d7caba9e046e
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_26256\3613356961.py:26: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., step=...) instead.
  "learning_rate": trial.suggest_discrete_uniform("learning_rate", 0.001, 0.05, 0.005),
c:\Users\Bayu Adjie Sidharta\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\distributions.py:685: UserWarning: The distribution is specified by [0.001, 0.05] and step=0.005, but the range is not divisible by `step`. It will be replaced by [0.001, 0.046].
  warnings.warn(
C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_26256\3613356961.py:28: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com

{'objective': 'RMSE', 'learning_rate': 0.001, 'depth': 16, 'l2_leaf_reg': 2.0, 'use_best_model': True, 'random_seed': 0}


[W 2023-06-08 14:38:28,381] Trial 0 failed with parameters: {'objective': 'RMSE', 'learning_rate': 0.001, 'depth': 16, 'l2_leaf_reg': 2.0} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "c:\Users\Bayu Adjie Sidharta\AppData\Local\Programs\Python\Python310\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Bayu Adjie Sidharta\AppData\Local\Temp\ipykernel_26256\3613356961.py", line 53, in objective
    model.fit(train_pool, eval_set=val_pool)
  File "c:\Users\Bayu Adjie Sidharta\AppData\Local\Programs\Python\Python310\lib\site-packages\catboost\core.py", line 5730, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline,
  File "c:\Users\Bayu Adjie Sidharta\AppData\Local\Programs\Python\Python310\lib\site-packages\catboost\core.py", line 2355, in _fit
    self._train(
  File "c:\Users\Bayu 

KeyboardInterrupt: 

In [43]:
from winsound import Beep

Beep(300, 200)
Beep(300, 200)
Beep(300, 700)

In [8]:
import Utils as utils
from catboost import CatBoostRegressor
from catboost import Pool

# utils.evaluate_catboost(dfX,y,kf)
def cbpred(dfX,y,kf):
    vals = []
    preds = []
    cbr = CatBoostRegressor()
    for train_index, test_index in kf.split(dfX):
    # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = dfX.iloc[train_index], dfX.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        train_pool = Pool(data=X_train, label=y_train)
        test_pool = Pool(data=X_test, label=y_test)

        
        cbr.fit(train_pool, verbose=False)
        cbr_pred = cbr.predict(test_pool)
        print(cbr.get_all_params())
        preds.append(cbr_pred)
        vals.append(y_test)

    return utils.error_rate("Catboost",flatten(preds),flatten(vals))
cols = ['Umur', 'Umur Squared', 'APM SMA', 'Pengeluaran Bulanan','Indeks Pembangunan Manusia', 'DPT', 'Sentiment', 'Weighted Sentiment']

res = cbpred(dfX[cols],y,kf)


{'nan_mode': 'Min', 'eval_metric': 'RMSE', 'iterations': 1000, 'sampling_frequency': 'PerTree', 'leaf_estimation_method': 'Newton', 'grow_policy': 'SymmetricTree', 'penalties_coefficient': 1, 'boosting_type': 'Plain', 'model_shrink_mode': 'Constant', 'feature_border_type': 'GreedyLogSum', 'bayesian_matrix_reg': 0.10000000149011612, 'eval_fraction': 0, 'force_unit_auto_pair_weights': False, 'l2_leaf_reg': 3, 'random_strength': 1, 'rsm': 1, 'boost_from_average': True, 'model_size_reg': 0.5, 'pool_metainfo_options': {'tags': {}}, 'subsample': 1, 'use_best_model': False, 'random_seed': 0, 'depth': 6, 'posterior_sampling': False, 'border_count': 254, 'classes_count': 0, 'auto_class_weights': 'None', 'sparse_features_conflict_fraction': 0, 'leaf_estimation_backtracking': 'AnyImprovement', 'best_model_min_trees': 1, 'model_shrink_rate': 0, 'min_data_in_leaf': 1, 'loss_function': 'RMSE', 'learning_rate': 0.028226999565958977, 'score_function': 'Cosine', 'task_type': 'CPU', 'leaf_estimation_ite

In [9]:
res

('Catboost', 0.03512429844587696, 0.028615520062230004, 0.47568209883632273)